In [1]:
from segment2d import *
import numpy as np
import csv
from matplotlib import pyplot as plt
from ipywidgets import interact
# visualize the image and mask in z ax is using interact, image and mask are in one slice
import SimpleITK as sitk
import scipy.ndimage as ndimage
from metrics_segmentation import hd

In [2]:
def preprocess_data(image_path):
    data = {}
    image = nib.load(image_path)
    data["header"] = image.header
    image = image.get_fdata()
    image = min_max_normalize(image)

    padded_image, crop_index, padded_index = pad_background(image, dim2pad=cfg.DATA.DIM2PAD)
    # padded_mask = pad_background_with_index(mask, crop_index, padded_index, dim2pad=cfg.DATA.DIM2PAD)
    data["crop_index"] = crop_index
    data["padded_index"] = padded_index
    data["original_shape"] = image.shape
    batch_images = []
    for i in range(padded_image.shape[-1]):
        slice_inputs = padded_image[..., i : i + 1]  # shape (224, 224, 1)
        slices_image = torch.from_numpy(slice_inputs.transpose(-1, 0, 1))  # shape (1, 224, 224)
        batch_images.append(slices_image)

    batch_images = torch.stack(batch_images).float()  # shape (9,1, 224, 224)
    data["image"] = batch_images
    return data


def predict_data(data, segmenter, patient="P", mvo=True, task="train_combine"):
    probability_output = segmenter.predict_patches(data["image"])  # shape (n, 5, 128, 128)
    seg = np.argmax(probability_output, axis=1).transpose(1, 2, 0)  # shape (128, 128, n)
    seg = remove_small_elements(seg, min_size_remove=800)

    myo = np.sum(seg == 2) + np.sum(seg == 3) + np.sum(seg == 4)
    infarction = np.sum(seg == 3) + np.sum(seg == 4)
    frequency_infarction = infarction / myo

    if patient == "N" and frequency_infarction < 0.015:
        seg[seg == 3] = 2
        seg[seg == 4] = 2
    elif patient == "P" and not mvo:
        seg[seg == 4] = 3

    if task == "train_combine":
        seg[seg == 4] = 3
    invert_seg = invert_padding(data["original_shape"], seg, data["crop_index"], data["padded_index"])
    return invert_seg


def make_volume(ndarray, voxel_spacing):
    volume = np.prod(voxel_spacing) * (ndarray.sum())
    return volume

# combine MI + PMO

In [31]:
task = "train_combine"
num_classes = 4 if task == "train_combine" else 5
with open("./test.csv", mode="r") as f:
    reader = csv.DictReader(f)
    list_test_subject = [row["path"] for row in reader]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCDenseNet(in_channels=cfg.DATA.INDIM_MODEL, n_classes=num_classes)
list_mask_test_dataset = [x.replace("Images", "Contours") for x in list_test_subject]
test_dataset = EMIDEC_Test_Loader(list_test_subject)
segmenter = Segmenter(
    model,
    cfg.DATA.CLASS_WEIGHT,
    5,
    0.001,
    0.5,
    50,
)
segmenter.eval()
if task == "train_combine":
    checkpoint = "./weights_train_combine/dice_0.7721.ckpt"
else:
    checkpoint = "./weights_train_full/myo_0.9266.ckpt"
segmenter = Segmenter.load_from_checkpoint(
    checkpoint_path=checkpoint,
    model=model,
    class_weight=cfg.DATA.CLASS_WEIGHT,
    num_classes=num_classes,
    learning_rate=0.001,
    factor_lr=0.5,
    patience_lr=50,
)
segmenter = segmenter.to(device)

In [32]:
MI_test_pts = [
    "Case_P050",
    "Case_P087",
    "Case_P001",
    "Case_P010",
    "Case_P017",
    "Case_P029",
    "Case_P090",
    "Case_P038",
    "Case_N052",
    "Case_N016",
    "Case_P100",
    "Case_P043",
    "Case_P051",
    "Case_N030",
    "Case_P007",
    "Case_P088",
    "Case_N025",
    "Case_P076",
    "Case_N046",
    "Case_N054",
    "Case_N049",
    "Case_N041",
    "Case_N023",
    "Case_P026",
    "Case_P031",
    "Case_N024",
    "Case_P064",
    "Case_P021",
    "Case_P015",
    "Case_P094",
]
task = "train_combine"
num_classes = 4 if task == "train_combine" else 5
with open("./test.csv", mode="r") as f:
    reader = csv.DictReader(f)
    list_test_subject = [row["path"] for row in reader]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCDenseNet(in_channels=cfg.DATA.INDIM_MODEL, n_classes=num_classes)
list_mask_test_dataset = [x.replace("Images", "Contours") for x in list_test_subject]
test_dataset = EMIDEC_Test_Loader(list_test_subject)
segmenter = Segmenter(
    model,
    cfg.DATA.CLASS_WEIGHT,
    5,
    0.001,
    0.5,
    50,
)
segmenter.eval()
if task == "train_combine":
    checkpoint = "./weights_train_combine/dice_0.7721.ckpt"
else:
    checkpoint = "./weights_train_full/myo_0.9266.ckpt"
segmenter = Segmenter.load_from_checkpoint(
    checkpoint_path=checkpoint,
    model=model,
    class_weight=cfg.DATA.CLASS_WEIGHT,
    num_classes=num_classes,
    learning_rate=0.001,
    factor_lr=0.5,
    patience_lr=50,
)
segmenter = segmenter.to(device)

In [33]:
dice_scores = {"dice_myocardium": [], "dice_lv": [], "dice_mi": []}
dice_scores_combined = {"dice_myocardium": [], "dice_lv": []}

for i in range(len(list_test_subject)):
    id_patient = list_test_subject[i].split("/")[-3]
    # if id_patient not in ["Case_N023"]:
    if id_patient not in MI_test_pts:
        continue
    test_image = nib.load(list_test_subject[i]).get_fdata()
    mask_image = nib.load(list_test_subject[i].replace("Images", "Contours")).get_fdata()
    mask_image[mask_image == 4] = 3
    data = preprocess_data(list_test_subject[i])
    seg = predict_data(data, segmenter, task=task).astype(np.uint8)
    dice_myo = dice_volume(mask_image, seg, class_index=2)
    dice_lv = dice_volume(mask_image, seg, class_index=1)
    dice_mi = dice_volume(mask_image, seg, class_index=3)
    # print("number of MI: ", np.sum(seg==3))
    dice_scores["dice_myocardium"].append(dice_myo)
    dice_scores["dice_lv"].append(dice_lv)
    dice_scores["dice_mi"].append(dice_mi)

    seg_combined = seg.copy()
    seg_combined[seg_combined == 3] = 2
    mask_combined = mask_image.copy()
    mask_combined[mask_combined == 3] = 2

    dice_myo_combined = dice_volume(mask_combined, seg_combined, class_index=2)
    dice_lv_combined = dice_volume(mask_combined, seg_combined, class_index=1)
    dice_scores_combined["dice_myocardium"].append(dice_myo_combined)
    dice_scores_combined["dice_lv"].append(dice_lv_combined)

    print(f"{id_patient} myo: {dice_myo:0.4f}, lv: {dice_lv:0.4f}, mi: {dice_mi:0.4f}")
    # print(f"patient {id_patient} dice myo: {dice_myo_combined:0.4f}, dice lv: {dice_lv_combined:0.4f}")
    # print(
    #     f"patient {id_patient} dice myo: {dice_myo_disease:0.4f}, dice lv: {dice_lv_disease:0.4f}, dice mi: {dice_mi_disease:0.4f}"
    # )

TypeError: predict_data() got an unexpected keyword argument 'threshold'

In [31]:
# calculate mean dice
for keys in dice_scores.keys():
    print(f"mean dice {keys}: {np.mean(dice_scores[keys]):0.4f}")

mean dice dice_myocardium: 0.8321
mean dice dice_lv: 0.9368
mean dice dice_mi: 0.7060


In [ ]:
interact(lambda z: plot_image_mask_z(test_image, mask_image, z, seg), z=(0, test_image.shape[-1] - 1))

interactive(children=(IntSlider(value=3, description='z', max=7), Output()), _dom_classes=('widget-interact',)…

<function __main__.<lambda>(z)>

# Full class

In [4]:

with open("./test.csv", mode="r") as f:
    reader = csv.DictReader(f)
    list_test_subject = [row["path"] for row in reader]
import os
# create folder for prediction
os.makedirs("./prediction", exist_ok=True)

In [5]:
task = "train_full"
cfg.DATA.CLASS_WEIGHT = [0.1, 2, 2, 17, 140]
num_classes = 4 if task == "train_combine" else 5
test_dataset = EMIDEC_Test_Loader(list_test_subject)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FCDenseNet(in_channels=cfg.DATA.INDIM_MODEL, n_classes=num_classes)
segmenter = Segmenter(
    model,
    cfg.DATA.CLASS_WEIGHT,
    5,
    0.001,
    0.5,
    50,
)
segmenter.eval()
if task == "train_combine":
    checkpoint = "./weights_train_combine/dice_0.7721.ckpt"
else:
    # checkpoint = "./weights_train_full/dice_0.7282.ckpt"
    checkpoint = "./weights_train_full/last.ckpt"
segmenter = Segmenter.load_from_checkpoint(
    checkpoint_path=checkpoint,
    model=model,
    class_weight=cfg.DATA.CLASS_WEIGHT,
    num_classes=num_classes,
    learning_rate=0.001,
    factor_lr=0.5,
    patience_lr=50,
)
segmenter = segmenter.to(device)

In [6]:
wrong_disease = ['Case_P087', 'Case_P010', 'Case_P017', 'Case_P100', 'Case_P051', 'Case_P026', 'Case_P031']
wrong_MVO = ["Case_P021", "Case_P015"]

In [7]:
dice_scores = {"dice_myocardium": [], "dice_lv": [], "dice_mi": [], "dice_mvo": []}
dice_scores_no_radio = {"dice_myocardium": [], "dice_lv": [], "dice_mi": [], "dice_mvo": []}

for i  in range(len(list_test_subject)):
    id_patient = list_test_subject[i].split("/")[-3]
    test_image = nib.load(list_test_subject[i]).get_fdata()
    mask_image = nib.load(list_test_subject[i].replace("Images", "Contours")).get_fdata()
    affine = nib.load(list_test_subject[i]).affine
    header = nib.load(list_test_subject[i]).header
    if "N" in id_patient:
        patient = "N"
    else:
        patient = "P"
    if id_patient in wrong_disease:
        patient = "N"
    is_MVO = False if np.sum(mask_image == 4) == 0 else True
    if id_patient in wrong_MVO:
        is_MVO = False

    data = preprocess_data(list_test_subject[i])
    seg = predict_data(data, segmenter, patient=patient, mvo=is_MVO, task=task).astype(np.uint8)
    seg_no_radio = predict_data(data, segmenter, patient="P", mvo=True, task=task).astype(np.uint8)
    # save segmentation result to nii file in prediction folder
    # create nii file from data["header"] and affine
    seg_nii = nib.Nifti1Image(seg, affine=affine, header=header)
    seg_no_radio_nii = nib.Nifti1Image(seg_no_radio, affine=affine, header=header)
    nib.save(seg_nii, f"./prediction/{id_patient}.nii.gz")
    nib.save(seg_no_radio_nii, f"./prediction/{id_patient}_no_radio.nii.gz")
    # if id_patient != "Case_P094":
    #     continue

    # print("number of MI: ", np.sum(seg == 4)+np.sum(seg==3))

    dice_myo = dice_volume(mask_image, seg, class_index=2)
    dice_lv = dice_volume(mask_image, seg, class_index=1)
    dice_mi = dice_volume(mask_image, seg, class_index=3)
    dice_mvo = dice_volume(mask_image, seg, class_index=4)
    dice_scores["dice_myocardium"].append(dice_myo)
    dice_scores["dice_lv"].append(dice_lv)
    dice_scores["dice_mi"].append(dice_mi)
    dice_scores["dice_mvo"].append(dice_mvo)
    
    dice_myo_no_radio = dice_volume(mask_image, seg_no_radio, class_index=2)
    dice_lv_no_radio = dice_volume(mask_image, seg_no_radio, class_index=1)
    dice_mi_no_radio = dice_volume(mask_image, seg_no_radio, class_index=3)
    dice_mvo_no_radio = dice_volume(mask_image, seg_no_radio, class_index=4)
    dice_scores_no_radio["dice_myocardium"].append(dice_myo_no_radio)
    dice_scores_no_radio["dice_lv"].append(dice_lv_no_radio)
    dice_scores_no_radio["dice_mi"].append(dice_mi_no_radio)
    dice_scores_no_radio["dice_mvo"].append(dice_mvo_no_radio)

    print(f"{id_patient} myo: {dice_myo:0.4f}, lv: {dice_lv:0.4f}, mi: {dice_mi:0.4f}, mvo: {dice_mvo:0.4f}")
    # print(f"patient {id_patient} dice myo: {dice_myo_combined:0.4f}, dice lv: {dice_lv_combined:0.4f}")
    # print(f"patient {id_patient} dice myo: {dice_myo_disease:0.4f}, dice lv: {dice_lv_disease:0.4f}, dice mi: {dice_mi_disease:0.4f}")

Case_P050 myo: 0.8013, lv: 0.9350, mi: 0.6877, mvo: 0.6499
Case_P087 myo: 0.8375, lv: 0.9071, mi: 0.5827, mvo: 1.0000
Case_P001 myo: 0.7436, lv: 0.9513, mi: 0.2592, mvo: 0.1108
Case_P010 myo: 0.8931, lv: 0.9322, mi: 0.4610, mvo: 0.0000
Case_P017 myo: 0.8291, lv: 0.9132, mi: 0.1804, mvo: 1.0000
Case_P029 myo: 0.8399, lv: 0.9427, mi: 0.5846, mvo: 0.4934
Case_P090 myo: 0.7803, lv: 0.9108, mi: 0.3300, mvo: 1.0000
Case_P038 myo: 0.8312, lv: 0.9464, mi: 0.6971, mvo: 1.0000
Case_N052 myo: 0.8254, lv: 0.9071, mi: 1.0000, mvo: 1.0000
Case_N016 myo: 0.8785, lv: 0.9190, mi: 1.0000, mvo: 1.0000
Case_P100 myo: 0.8917, lv: 0.9403, mi: 0.5996, mvo: 0.0000
Case_P043 myo: 0.8300, lv: 0.9440, mi: 0.5818, mvo: 0.1661
Case_P051 myo: 0.8202, lv: 0.8862, mi: 0.3221, mvo: 1.0000
Case_N030 myo: 0.8914, lv: 0.9311, mi: 1.0000, mvo: 1.0000
Case_P007 myo: 0.8481, lv: 0.9393, mi: 0.3830, mvo: 1.0000
Case_P088 myo: 0.8201, lv: 0.9131, mi: 0.2061, mvo: 1.0000
Case_N025 myo: 0.8795, lv: 0.9427, mi: 1.0000, mvo: 1.00

In [77]:
for keys in dice_scores.keys():
    print(f"mean dice {keys}: {np.mean(dice_scores[keys]):0.4f}")
    print(f"std dice {keys}: {np.std(dice_scores[keys]):0.4f}")

mean dice dice_myocardium: 0.8401
std dice dice_myocardium: 0.0401
mean dice dice_lv: 0.9312
std dice dice_lv: 0.0180
mean dice dice_mi: 0.6256
std dice dice_mi: 0.3072
mean dice dice_mvo: 0.7334
std dice dice_mvo: 0.4045


In [73]:
from scipy.stats import ttest_rel
import numpy as np

# Assuming you have two arrays of Dice scores for Model A and Model B
dice_scores_a = dice_scores["dice_mi"] 
dice_scores_b = dice_scores_no_radio["dice_mi"]

# Perform a paired t-test
t_stat, p_value = ttest_rel(dice_scores_a, dice_scores_b)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

t-statistic: 2.4083189157578895
p-value: 0.022608379396821858


In [74]:
dice_scores_a = dice_scores["dice_mvo"]
dice_scores_b = dice_scores_no_radio["dice_mvo"]

# Perform a paired t-test
t_stat, p_value = ttest_rel(dice_scores_a, dice_scores_b)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")

t-statistic: 0.9561324393399092
p-value: 0.34690852544902984


In [57]:
# write dice score to csv file
with open("dice_score.csv", mode="w") as f:
    writer = csv.writer(f)
    writer.writerow(["patient", "dice_myocardium", "dice_lv", "dice_mi", "dice_mvo"])
    for i in range(len(list_test_subject)):
        id_patient = list_test_subject[i].split("/")[-3]
        writer.writerow(
            [
                id_patient,
                dice_scores["dice_myocardium"][i],
                dice_scores["dice_lv"][i],
                dice_scores["dice_mi"][i],
                dice_scores["dice_mvo"][i],
            ]
        )
# read dice score from csv file
import pandas as pd

dice_score_df = pd.read_csv("dice_score.csv")
dice_score_df.describe()

In [138]:
id_patient = "Case_P050"
test_image = nib.load(f"./emidec-dataset-1.0.1/{id_patient}/Images/{id_patient}.nii.gz").get_fdata()
mask_image = nib.load(f"./emidec-dataset-1.0.1/{id_patient}/Contours/{id_patient}.nii.gz").get_fdata()
seg = nib.load(f"./prediction/{id_patient}.nii.gz").get_fdata()
padded_image, crop_index, padded_index = pad_background(test_image, dim2pad=cfg.DATA.DIM2PAD)
mask_padded = pad_background_with_index(mask_image, crop_index, padded_index, dim2pad=cfg.DATA.DIM2PAD)
seg_padded = pad_background_with_index(seg, crop_index, padded_index, dim2pad=cfg.DATA.DIM2PAD)


def plot_image_mask_z(image, mask, z, prediction=None):
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image[..., z], cmap="gray")
    ax[0].imshow(image[..., z], cmap="gray")
    ax[0].set_title("Prediction")
    # set color for jet cmap, pixel value 1 is red, 2 is green, 3 is blue, 4 is yellow
    ax[0].imshow(mask[..., z], cmap="jet", alpha=0.3, vmin=0, vmax=5)

    if prediction is not None:
        ax[1].imshow(image[..., z], cmap="gray")
        ax[1].set_title("Ground Truth")
        ax[1].imshow(mask[..., z], cmap="jet", alpha=0.3, vmin=0, vmax=5)
    # off axis
    ax[0].axis("off")
    ax[1].axis("off")
    plt.show()


interact(lambda z: plot_image_mask_z(padded_image, mask_padded, z, seg_padded), z=(0, test_image.shape[-1] - 1))

interactive(children=(IntSlider(value=2, description='z', max=5), Output()), _dom_classes=('widget-interact',)…

<function __main__.<lambda>(z)>

In [47]:
volumeDifferenceRate_infarction = []
volumeDifferenceRate_noreflow = []
volumeDifference_myo = []
volumeDifference_infarction = []
volumeDifference_noreflow = []
HD_myo = []

volume_prediction = {"volume_myo": [], "volume_infarction": [], "volume_noreflow": []}
volume_GT = {"volume_myo": [], "volume_infarction": [], "volume_noreflow": []}

volume_pred_rate = {"volume_myo": [], "volume_infarction": [], "volume_noreflow": []}
volume_GT_rate = {"volume_myo": [], "volume_infarction": [], "volume_noreflow": []}

for i in range(len(list_test_subject)):
    id_patient = list_test_subject[i].split("/")[-3]
    test_image = sitk.ReadImage(list_test_subject[i])
    test_image_array = sitk.GetArrayFromImage(test_image)
    mask = sitk.ReadImage(list_test_subject[i].replace("Images", "Contours"), sitk.sitkInt16)
    GTArray = sitk.GetArrayFromImage(mask)
    spacing = mask.GetSpacing()

    prediction = sitk.ReadImage(f"./prediction/{id_patient}_no_radio.nii.gz", sitk.sitkInt16)
    prediction = sitk.GetArrayFromImage(prediction)

    volume_myo_pred = (prediction ==2) + (prediction == 3) + (prediction == 4)

    volume_myo_volume = make_volume(volume_myo_pred, spacing)

    volume_infarction_pred = (prediction == 3) + (prediction == 4)
    volume_infarction_volume = make_volume(volume_infarction_pred, spacing)
    volume_noreflow_pred = (prediction == 4)
    volume_noreflow_volume= make_volume(volume_noreflow_pred, spacing)

    mask_myo = (GTArray == 2) + (GTArray == 3) + (GTArray == 4)
    mask_myo_volume = make_volume(mask_myo, spacing)
    mask_infarction = (GTArray == 3) + (GTArray == 4)
    mask_infarction_volume = make_volume(mask_infarction, spacing)
    mask_noreflow = (GTArray == 4)
    mask_noreflow_volume = make_volume(mask_noreflow, spacing)

    volume_prediction["volume_myo"].append(volume_myo_volume)
    volume_GT["volume_myo"].append(mask_myo_volume)

    if mask_infarction_volume != 0:
        volume_prediction["volume_infarction"].append(volume_infarction_volume)
        volume_GT["volume_infarction"].append(mask_infarction_volume)
        volume_pred_rate["volume_infarction"].append(volume_infarction_volume/volume_myo_volume)
        volume_GT_rate["volume_infarction"].append(mask_infarction_volume/mask_myo_volume)
    if mask_noreflow_volume != 0:
        volume_prediction["volume_noreflow"].append(volume_noreflow_volume)
        volume_GT["volume_noreflow"].append(mask_noreflow_volume)
        volume_pred_rate["volume_noreflow"].append(volume_noreflow_volume/volume_myo_volume)
        volume_GT_rate["volume_noreflow"].append(mask_noreflow_volume/mask_myo_volume)

    volumeDifference_myo.append(abs(volume_myo_volume - mask_myo_volume))
    volumeDifference_infarction.append(abs(volume_infarction_volume - mask_infarction_volume))
    volumeDifference_noreflow.append(abs(volume_noreflow_volume - mask_noreflow_volume))
    HD_myo.append(hd(volume_myo_pred, mask_myo, spacing))

    volumeDifferenceRate_infarction.append(abs(volume_infarction_volume - mask_infarction_volume) / mask_myo_volume)
    volumeDifferenceRate_noreflow.append(abs(volume_noreflow_volume - mask_noreflow_volume) / mask_myo_volume)

NiftiImageIO (0x58c604ddfc60): ./emidec-dataset-1.0.1/Case_P050/Images/Case_P050.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604ddfc60): ./emidec-dataset-1.0.1/Case_P050/Images/Case_P050.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604e26b90): ./emidec-dataset-1.0.1/Case_P050/Contours/Case_P050.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604e26b90): ./emidec-dataset-1.0.1/Case_P050/Contours/Case_P050.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604ddfc60): ./prediction/Case_P050_no_radio.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604ddfc60): ./prediction/Case_P050_no_radio.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604e26b90): ./emidec-dataset-1.0.1/Case_P087/Images/Case_P087.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604e26b90): ./emidec-dataset-1.0.1/Case_P087/Images/Case_P087.nii.gz has unexpected scales in sform

NiftiImageIO (0x58c604e26b90): ./prediction/Case_P087_no_radio.nii.gz ha

In [48]:
print(f"mean volume difference myo: {np.mean(volumeDifference_myo):0.4f}")
print(f"mean volume difference infarction: {np.mean(volumeDifference_infarction):0.4f}")
print(f"mean volume difference noreflow: {np.mean(volumeDifference_noreflow):0.4f}")
print(f"std volume difference myo: {np.std(volumeDifference_myo):0.4f}")
print(f"std volume difference infarction: {np.std(volumeDifference_infarction):0.4f}")
print(f"std volume difference noreflow: {np.std(volumeDifference_noreflow):0.4f}")


mean volume difference myo: 9362.7063
mean volume difference infarction: 8282.6288
mean volume difference noreflow: 1688.9607
std volume difference myo: 11371.4480
std volume difference infarction: 15413.0699
std volume difference noreflow: 6136.7082


In [49]:
print(f"mean volume difference rate infarction: {np.mean(volumeDifferenceRate_infarction):0.5f}")
print(f"mean volume difference rate noreflow: {np.mean(volumeDifferenceRate_noreflow):0.5f}")
print(f"std volume difference rate infarction: {np.std(volumeDifferenceRate_infarction):0.5f}")
print(f"std volume difference rate noreflow: {np.std(volumeDifferenceRate_noreflow):0.5f}")


mean volume difference rate infarction: 0.05949
mean volume difference rate noreflow: 0.01023
std volume difference rate infarction: 0.08144
std volume difference rate noreflow: 0.03131


In [84]:
volume_diff = {"volume_infarction": volumeDifference_infarction, "volume_noreflow": volumeDifference_noreflow}
volume_diff_rate = {"volume_infarction": volumeDifferenceRate_infarction, "volume_noreflow": volumeDifferenceRate_noreflow}

In [89]:
volume_diff_no_radio = {"volume_infarction": volumeDifference_infarction, "volume_noreflow": volumeDifference_noreflow}
volume_diff_rate_no_radio = {
    "volume_infarction": volumeDifferenceRate_infarction,
    "volume_noreflow": volumeDifferenceRate_noreflow,
}

In [50]:
from scipy.stats import shapiro
from scipy.stats import wilcoxon

# Check for normality in the difference between predictions and ground truths
differences = np.array(volume_prediction["volume_infarction"]) - np.array(volume_GT["volume_infarction"])
stat, p_value = shapiro(differences)

if p_value < 0.05:
    print("Differences are not normally distributed, use a non-parametric test.")
else:
    print("Differences are normally distributed, t-test is okay.")

stat, p_value = wilcoxon(volume_prediction["volume_infarction"], volume_GT["volume_infarction"])
print(f"Wilcoxon test for infarction volume: {p_value}")

Differences are not normally distributed, use a non-parametric test.
Wilcoxon test for infarction volume: 5.7220458984375e-06


In [51]:
# Check for normality in the difference between predictions and ground truths
differences = np.array(volume_pred_rate["volume_noreflow"]) - np.array(volume_GT_rate["volume_noreflow"])
stat, p_value = shapiro(differences)

if p_value < 0.05:
    print("Differences are not normally distributed, use a non-parametric test.")
else:
    print("Differences are normally distributed, t-test is okay.")

stat, p_value = wilcoxon(volume_prediction["volume_infarction"], volume_GT["volume_infarction"])
print(f"Wilcoxon test for infarction volume: {p_value}")

Differences are not normally distributed, use a non-parametric test.
Wilcoxon test for infarction volume: 5.7220458984375e-06


In [11]:
volume_prediction["volume_infarction"]

[38176.3916015625,
 2099.609375,
 18603.515625,
 4678.819699419872,
 1680.121619337136,
 26222.16796875,
 6741.753839618635,
 5541.9921875,
 0.0,
 0.0,
 9409.130859375,
 16136.71875,
 1254.7743739353295,
 0.0,
 2637.152921491201,
 8149.4384765625,
 0.0,
 29949.8291015625,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6411.80419921875,
 1850.9765625,
 0.0,
 1079.736328125,
 3232.6390650537305,
 30206.752132342936,
 30505.37109375]

In [82]:
print(f"mean HD myo: {np.mean(HD_myo):0.4f}")
print(f"std HD myo: {np.std(HD_myo):0.4f}")

mean HD myo: 15.7189
std HD myo: 7.4393


In [83]:
for keys in dice_scores.keys():
    print(f"mean dice {keys}: {np.mean(dice_scores[keys]):0.4f}")

mean dice dice_myocardium: 0.8401
mean dice dice_lv: 0.9312
mean dice dice_mi: 0.6256
mean dice dice_mvo: 0.7334


In [61]:
for keys in dice_scores_combined.keys():
    print(f"mean dice {keys}: {np.mean(dice_scores_combined[keys]):0.4f}")

mean dice dice_myocardium: 0.8374
mean dice dice_lv: 0.9313


In [56]:
for keys in dice_scores_disease.keys():
    print(f"mean dice {keys}: {np.mean(dice_scores_disease[keys]):0.4f}")

mean dice dice_myocardium: 0.8248
mean dice dice_lv: 0.9314
mean dice dice_MI: 0.5071


ValueError: could not broadcast input array from shape (10,128,0) into shape (10,128,2)